# 데이터

- BatchClientExternal class, `generate_prompt_external.json` 사용

- 외부 크롤링 데이터 스키마 (컬럼: title, contents, url, date 등)

## 데이터셋 불러오기

In [ ]:
import pandas as pd
data = pd.read_json('postprocessed_data/jisikin.json')
data.head()

In [ ]:
data.shape

In [3]:
# 작성일 열을 datetime으로 변환
data['date'] = pd.to_datetime(data['date'], format='%y-%m-%d')

In [4]:
from datetime import datetime

# 2020년 1월 1일 이후의 데이터만 필터링
cutoff_date = datetime(2020, 1, 1)
filtered_data = data[data['date'] >= cutoff_date]

In [ ]:
filtered_data = filtered_data.sort_values(by="date", ascending=False)
filtered_data.reset_index(drop=True, inplace=True)
filtered_data

In [ ]:
filtered_data.shape

## 데이터셋 생성

저장 형식:

Document (
- 상담내용 : raw_text (str), Embedding(list)
- 해결방법 (str) : Answer
- 내담자_추가정보 (list): raw_text (str), Embedding(list)
- metadata {
    date : raw_text(str)
}
)


In [7]:
import pandas as pd
from typing import Dict, List
import json

def transform_dataframe_to_documents(df: pd.DataFrame) -> List[Dict]:
    """
    데이터프레임을 지정된 문서 형식으로 변환

    Args:
        df: 입력 데이터프레임

    Returns:
        변환된 문서 리스트
    """
    documents = []

    for _, row in df.iterrows():
        document = {
            "상담내용": {
                "raw_text": row["title"],
                "Embedding": []
            },
            "내담자_추가정보": {
                "raw_text": [],
                "Embedding": []  # LLM 추출 부분 빈 리스트로 초기화
            },
            "해결방법": {
                "raw_text": row["contents"],
                "Embedding": []
            },
            "metadata": {
                "url": row["url"],
                "date": str(row["date"])
            }
        }
        documents.append(document)

    return documents


In [ ]:
jisikin_2020 = transform_dataframe_to_documents(filtered_data)
jisikin_2020

In [ ]:
jisikin_2020

In [10]:
def save_documents_to_jsonl(documents: List[Dict], file_path: str):
    """
    문서 리스트를 JSONL 파일로 저장

    Args:
        documents: 문서 딕셔너리 리스트
        file_path: 저장할 파일 경로
    """
    with open(file_path, 'w', encoding='utf-8') as f:
        for item in documents:
            f.write(json.dumps(item, ensure_ascii=False) + '/n')

In [11]:
save_documents_to_jsonl(jisikin_2020, '../data/jisikin_filtered.jsonl')

In [ ]:
filtered_data.head()

In [ ]:
len(filtered_data)

In [ ]:
from BatchClient import BatchClientExternal
import utils

batch_client = BatchClientExternal(config_path="../configs/config.json")

In [15]:
FILE_YEAR = "2020_2024"

In [ ]:
filtered_data.columns

### 내담자정보 - 추가정보

In [ ]:
import time
metadata = batch_client.make_generate(df=filtered_data,
                                      prompt_path="../prompts/generate_prompt_external.json",
                                      batch_jsonl_path=f"../data/batch_jsonl/input/jisikin_consultation_{FILE_YEAR}_내담자정보_batch.jsonl",
                                      meta_path=f"../data/meta_csv/jisikin_consultation_{FILE_YEAR}_meta.csv")

request_id =  metadata.id
print('## request_id : ', request_id)

while True:
    status = batch_client.get_batch_status(batch_id=metadata.id)
    print('## current status : ', status.status)

    if status.status == "completed":
        output_file_id = status.output_file_id
        print(f'## batch completed!/noutput file id : {output_file_id}')
        break

    else:
        print('## Wait untill completed...')
        time.sleep(60)

In [ ]:
batch_client.get_batch_answer(output_file_id=output_file_id, result_path=f"../data/batch_jsonl/output/jisikin_consultation_{FILE_YEAR}_내담자정보_batch.jsonl")

In [1]:
FILE_YEAR = "2020_2024"

In [ ]:
import utils

utils.get_external_result_csv(gpt_answer_path=f"batch_jsonl/output/jisikin_consultation_{FILE_YEAR}_내담자정보_batch.jsonl",
                            meta_path=f"meta_csv/jisikin_consultation_{FILE_YEAR}_meta.csv",
                            output_path=f"postprocessed_data/jisikin_consultation_{FILE_YEAR}.csv",
                            return_type = "generate",
                            column_name="내담자정보"
)

In [ ]:
utils.get_result_jsonl(meta_path=f"meta_csv/jisikin_consultation_{FILE_YEAR}_meta.csv", result_jsonl_path=f"batch_jsonl/output/jisikin_consultation_{FILE_YEAR}_batch.jsonl")

# 대한법률구조공단

## 데이터셋 불러오기

In [1]:
import pandas as pd
data = pd.read_json('postprocessed_data/klac.json')
data.head()


,title,contents,url
0,노동근로일반\n외국인회사의 경우에도 근로기준법이 적용되는지\n저는 외국인이 경영하는...,「근로기준법」제11조 제1항은 “근로기준법은 상시 5인 이상의 근로자를 사용하는 모...,https://www.klac.or.kr/
1,노동근로일반\n외국인회사의 경우에도 근로기준법이 적용되는지\n외국인이 경영하는 상시...,근로기준법 제11조 제1항은 “근로기준법은 상시 5인 이상의 근로자를 사용하는 모든...,https://www.klac.or.kr/
2,노동산업재해\n불법체류외국인근로자도 산업재해보상보험법상 요양급여지를 받을 수 있는지...,이와 같은 사례에서 대법원 판례는 “외국인이 취업자격이 아닌 산업연수 체류자격으로 ...,https://www.klac.or.kr/
3,주택임대차적용범위\n외국인의 경우도 주택임대차보호법이 적용되는지\n저는 러시아 국적...,주택임대차보호법 제1조는 “이 법은 주거용 건물의 임대차(賃貸借)에 관하여 「민법」...,https://www.klac.or.kr/
4,주택임대차대항력\n재외국민이 주택을 임대차계약을 체결하고 그 동거가족인 외국인 또는...,"구「출입국관리법」(2010.5.14.법률 제10282호로 개정되기 전의 것,이하 ‘...",https://www.klac.or.kr/


In [2]:
data.shape

(29, 3)

## 데이터셋 생성
저장 형식:

Document (
- 상담내용 : raw_text (str), Embedding(list)
- 해결방법 (str) : Answer
- 내담자_추가정보 (list): raw_text (str), Embedding(list)
- metadata {
    url : raw_text (str)
}
)


In [3]:
import pandas as pd
from typing import Dict, List
import json

def transform_dataframe_to_documents(df: pd.DataFrame) -> List[Dict]:
    """
    데이터프레임을 지정된 문서 형식으로 변환

    Args:
        df: 입력 데이터프레임

    Returns:
        변환된 문서 리스트
    """
    documents = []

    for _, row in df.iterrows():
        document = {
            "상담내용": {
                "raw_text": row["title"],
                "Embedding": []
            },
            "내담자_추가정보": {
                "raw_text": [],
                "Embedding": []  # LLM 추출 부분 빈 리스트로 초기화
            },
            "해결방법": {
                "raw_text": row["contents"],
                "Embedding": []
            },
            "metadata": {
                "url": row["url"]
            }
        }
        documents.append(document)

    return documents


In [4]:
klac = transform_dataframe_to_documents(data)
klac

[{'상담내용': {'raw_text': '노동근로일반\n외국인회사의 경우에도 근로기준법이 적용되는지\n저는 외국인이 경영하는 상시 근로자 10명인 회사에서 근무하고 있습니다. 이 경우에도 「근로기준법」이 적용되는지? 구체적인 적용기준은 어떤지요?',
   'Embedding': []},
  '내담자_추가정보': {'raw_text': [], 'Embedding': []},
  '해결방법': {'raw_text': '「근로기준법」제11조 제1항은 “근로기준법은 상시 5인 이상의 근로자를 사용하는 모든 사업 또는 사업장에 적용한다.”라고 규정하고 있고, 이는 국가·지방자치단체에 의한 사업은 물론 국영기업체, 공익사업체, 사회사업단체, 종교단체가 행하는 모든 사업에 적용됩니다(서울지법 1996. 9. 10. 선고 96가단90373 판결, 대법원 1979. 3. 27. 선고 78다163 판결).\n이와 같이 「근로기준법」은 이 법의 적용대상이 되는 사업을 한정시키지 않고 모든 사업에 적용되도록 규정하고 있으므로 사업의 개념은 탄력적으로 파악되어야 할 것입니다.\n여기서 사업 또는 사업장의 판단기준과 관련하여 문제되는 것은 다음과 같습니다.\n첫째, 한 사업주가 2개 이상의 사업장을 갖는 경우에 장소적으로 분산되어 있다 하여도 동일한 조직적 관련을 가지고 사업의 독립성이 없을 경우에는 하나의 사업장으로 보아야 할 것입니다.\n둘째, 계절적 사업 또는 일용근로자만을 사용하는 사업장의 경우도 같은 법의 적용을 받아야 합니다(대법원 1987. 4. 14. 선고 87도 153 판결).\n셋째, 외국인사업에 대해서는 외국인회사가 우리나라에서 한국인 노동자를 고용하여 사용자의 지위를 가지는 한, 자연인이나 법인을 불문하고 근로기준법상의 규정 및 의무를 준수·이행하여야 합니다.\n넷째, 「근로기준법」도 국내법으로서 국내에서만 적용되며 통치권이 미치지 못하는 국외의 사업장에 대해서는 적용되지 않는 것이 원칙입니다. 그러므로 한국인이 경영하는 외국소재 기업체에서는 근로기준법이 적용되

In [5]:
def save_documents_to_jsonl(documents: List[Dict], file_path: str):
    """
    문서 리스트를 JSONL 파일로 저장

    Args:
        documents: 문서 딕셔너리 리스트
        file_path: 저장할 파일 경로
    """
    with open(file_path, 'w', encoding='utf-8') as f:
        for item in documents:
            f.write(json.dumps(item, ensure_ascii=False) + '/n')


In [6]:
save_documents_to_jsonl(klac, '../data/klac.jsonl')

In [7]:
from BatchClient import BatchClientExternal
import utils

batch_client = BatchClientExternal(config_path="../configs/config.json")
data.columns

c:\Users\hwyew\miniconda3\envs\hello\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


## config loaded ##
## client loaded ##


Index(['title', 'contents', 'url'], dtype='object')

In [8]:
### 내담자정보 - 추가정보
import time
metadata = batch_client.make_generate(df=data,
                                      prompt_path="../prompts/generate_prompt_external.json",
                                      batch_jsonl_path=f"../data/batch_jsonl/input/klac_내담자정보_batch.jsonl",
                                      meta_path=f"../data/meta_csv/klac_meta.csv")

request_id =  metadata.id
print('## request_id : ', request_id)

## Generate model loaded
## jsonl file saved in ../data/batch_jsonl/input/klac_내담자정보_batch.jsonl
## meta file saved in ../data/meta_csv/klac_meta.csv
## request_id :  batch_6776a6fb68bc8190bb7d567bb9f45433


In [9]:
while True:
    status = batch_client.get_batch_status(batch_id=metadata.id)
    print('## current status : ', status.status)

    if status.status == "completed":
        output_file_id = status.output_file_id
        print(f'## batch completed!/noutput file id : {output_file_id}')
        break

    else:
        print('## Wait untill completed...')
        time.sleep(60)

## current status :  in_progress
## Wait untill completed...
## current status :  completed
## batch completed!/noutput file id : file-C3NYbDzS8sTB4oSFyxSE4M


In [10]:
batch_client.get_batch_answer(output_file_id=output_file_id, result_path=f"../data/batch_jsonl/output/klac_내담자정보_batch.jsonl")

## file saved in ../data/batch_jsonl/output/klac_내담자정보_batch.jsonl


In [13]:
import utils

utils.get_external_result_csv(gpt_answer_path=f"batch_jsonl/output/klac_내담자정보_batch.jsonl",
                            meta_path=f"meta_csv/klac_meta.csv",
                            output_path=f"postprocessed_data/klac.csv",
                            return_type = "generate",
                            column_name="내담자정보"
)
utils.get_result_jsonl(meta_path=f"postprocessed_data/klac.csv", result_jsonl_path=f"batch_jsonl/output/klac_batch.jsonl")

## result saved in postprocessed_data/klac.csv
## jsonl file saved in batch_jsonl/output/klac_batch.jsonl


# 후처리 (01.06)

상담 사례 2개 가져오기
- 내담자 추가정보 -> title
- 상담내용 + 해결방법 -> contents
- metadata 존치

### klac

In [3]:
import json

# JSONL 파일 경로
# data\batch_jsonl\output\klac_batch.jsonl
file_path = "./batch_jsonl/output/klac_batch.jsonl"

# JSONL 파일 읽기
data = []
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        # 각 라인을 JSON으로 파싱
        record = json.loads(line)
        data.append(record)

# 읽어온 데이터 확인
for entry in data:
    print(entry)

{'상담제목': {'raw_text': '노동근로일반\n외국인회사의 경우에도 근로기준법이 적용되는지\n저는 외국인이 경영하는 상시 근로자 10명인 회사에서 근무하고 있습니다. 이 경우에도 「근로기준법」이 적용되는지? 구체적인 적용기준은 어떤지요?', 'Embedding': []}, '내담자_추가정보': '\n외국인이 경영하는 회사에서 근무 중\n상시 근로자 10명인 회사\n', '해결방법': {'raw_text': '「근로기준법」제11조 제1항은 “근로기준법은 상시 5인 이상의 근로자를 사용하는 모든 사업 또는 사업장에 적용한다.”라고 규정하고 있고, 이는 국가·지방자치단체에 의한 사업은 물론 국영기업체, 공익사업체, 사회사업단체, 종교단체가 행하는 모든 사업에 적용됩니다(서울지법 1996. 9. 10. 선고 96가단90373 판결, 대법원 1979. 3. 27. 선고 78다163 판결).\n이와 같이 「근로기준법」은 이 법의 적용대상이 되는 사업을 한정시키지 않고 모든 사업에 적용되도록 규정하고 있으므로 사업의 개념은 탄력적으로 파악되어야 할 것입니다.\n여기서 사업 또는 사업장의 판단기준과 관련하여 문제되는 것은 다음과 같습니다.\n첫째, 한 사업주가 2개 이상의 사업장을 갖는 경우에 장소적으로 분산되어 있다 하여도 동일한 조직적 관련을 가지고 사업의 독립성이 없을 경우에는 하나의 사업장으로 보아야 할 것입니다.\n둘째, 계절적 사업 또는 일용근로자만을 사용하는 사업장의 경우도 같은 법의 적용을 받아야 합니다(대법원 1987. 4. 14. 선고 87도 153 판결).\n셋째, 외국인사업에 대해서는 외국인회사가 우리나라에서 한국인 노동자를 고용하여 사용자의 지위를 가지는 한, 자연인이나 법인을 불문하고 근로기준법상의 규정 및 의무를 준수·이행하여야 합니다.\n넷째, 「근로기준법」도 국내법으로서 국내에서만 적용되며 통치권이 미치지 못하는 국외의 사업장에 대해서는 적용되지 않는 것이 원칙입니다. 그러므로 한국인이 경영하는 외국소재 기업체에서는 근로기준법이 적용되

In [4]:
data[0]

{'상담제목': {'raw_text': '노동근로일반\n외국인회사의 경우에도 근로기준법이 적용되는지\n저는 외국인이 경영하는 상시 근로자 10명인 회사에서 근무하고 있습니다. 이 경우에도 「근로기준법」이 적용되는지? 구체적인 적용기준은 어떤지요?',
  'Embedding': []},
 '내담자_추가정보': '\n외국인이 경영하는 회사에서 근무 중\n상시 근로자 10명인 회사\n',
 '해결방법': {'raw_text': '「근로기준법」제11조 제1항은 “근로기준법은 상시 5인 이상의 근로자를 사용하는 모든 사업 또는 사업장에 적용한다.”라고 규정하고 있고, 이는 국가·지방자치단체에 의한 사업은 물론 국영기업체, 공익사업체, 사회사업단체, 종교단체가 행하는 모든 사업에 적용됩니다(서울지법 1996. 9. 10. 선고 96가단90373 판결, 대법원 1979. 3. 27. 선고 78다163 판결).\n이와 같이 「근로기준법」은 이 법의 적용대상이 되는 사업을 한정시키지 않고 모든 사업에 적용되도록 규정하고 있으므로 사업의 개념은 탄력적으로 파악되어야 할 것입니다.\n여기서 사업 또는 사업장의 판단기준과 관련하여 문제되는 것은 다음과 같습니다.\n첫째, 한 사업주가 2개 이상의 사업장을 갖는 경우에 장소적으로 분산되어 있다 하여도 동일한 조직적 관련을 가지고 사업의 독립성이 없을 경우에는 하나의 사업장으로 보아야 할 것입니다.\n둘째, 계절적 사업 또는 일용근로자만을 사용하는 사업장의 경우도 같은 법의 적용을 받아야 합니다(대법원 1987. 4. 14. 선고 87도 153 판결).\n셋째, 외국인사업에 대해서는 외국인회사가 우리나라에서 한국인 노동자를 고용하여 사용자의 지위를 가지는 한, 자연인이나 법인을 불문하고 근로기준법상의 규정 및 의무를 준수·이행하여야 합니다.\n넷째, 「근로기준법」도 국내법으로서 국내에서만 적용되며 통치권이 미치지 못하는 국외의 사업장에 대해서는 적용되지 않는 것이 원칙입니다. 그러므로 한국인이 경영하는 외국소재 기업체에서는 근로기준법이

In [5]:
data_list = []

for row in data:
    data_list.append({"title" : row['내담자_추가정보'],
    "contents" : "[상담내용]\n" + row["상담제목"]["raw_text"] + "\n\n[해결방법]\n" + row["해결방법"]["raw_text"],
    "metadata" : {"url" : row['metadata']['url']}})

In [7]:
data_list[0]

{'title': '\n외국인이 경영하는 회사에서 근무 중\n상시 근로자 10명인 회사\n',
 'contents': '[상담내용]\n노동근로일반\n외국인회사의 경우에도 근로기준법이 적용되는지\n저는 외국인이 경영하는 상시 근로자 10명인 회사에서 근무하고 있습니다. 이 경우에도 「근로기준법」이 적용되는지? 구체적인 적용기준은 어떤지요?\n\n[해결방법]\n「근로기준법」제11조 제1항은 “근로기준법은 상시 5인 이상의 근로자를 사용하는 모든 사업 또는 사업장에 적용한다.”라고 규정하고 있고, 이는 국가·지방자치단체에 의한 사업은 물론 국영기업체, 공익사업체, 사회사업단체, 종교단체가 행하는 모든 사업에 적용됩니다(서울지법 1996. 9. 10. 선고 96가단90373 판결, 대법원 1979. 3. 27. 선고 78다163 판결).\n이와 같이 「근로기준법」은 이 법의 적용대상이 되는 사업을 한정시키지 않고 모든 사업에 적용되도록 규정하고 있으므로 사업의 개념은 탄력적으로 파악되어야 할 것입니다.\n여기서 사업 또는 사업장의 판단기준과 관련하여 문제되는 것은 다음과 같습니다.\n첫째, 한 사업주가 2개 이상의 사업장을 갖는 경우에 장소적으로 분산되어 있다 하여도 동일한 조직적 관련을 가지고 사업의 독립성이 없을 경우에는 하나의 사업장으로 보아야 할 것입니다.\n둘째, 계절적 사업 또는 일용근로자만을 사용하는 사업장의 경우도 같은 법의 적용을 받아야 합니다(대법원 1987. 4. 14. 선고 87도 153 판결).\n셋째, 외국인사업에 대해서는 외국인회사가 우리나라에서 한국인 노동자를 고용하여 사용자의 지위를 가지는 한, 자연인이나 법인을 불문하고 근로기준법상의 규정 및 의무를 준수·이행하여야 합니다.\n넷째, 「근로기준법」도 국내법으로서 국내에서만 적용되며 통치권이 미치지 못하는 국외의 사업장에 대해서는 적용되지 않는 것이 원칙입니다. 그러므로 한국인이 경영하는 외국소재 기업체에서는 근로기준법이 적용되지 않습니다. 반면, 국내에 본사(기업체)가 있고 그 출장소나

In [9]:
# JSON 파일로 저장
output_file = "./postprocessed_data/klac.json"
with open(output_file, 'w', encoding='utf-8') as file:
    json.dump(data_list, file, ensure_ascii=False, indent=4)

print(f"JSON 파일이 저장되었습니다: {output_file}")

JSON 파일이 저장되었습니다: ./postprocessed_data/klac.json


### jisikin

In [10]:
import json

# JSONL 파일 경로
# data\batch_jsonl\output\klac_batch.jsonl
file_path = "./batch_jsonl/output/jisikin_consultation_2020_2024_batch.jsonl"

# JSONL 파일 읽기
data = []
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        # 각 라인을 JSON으로 파싱
        record = json.loads(line)
        data.append(record)

# 읽어온 데이터 확인
for entry in data:
    print(entry)

{'상담제목': {'raw_text': '퇴직금 지급받을 irp통장 조건을 확실히 알고싶습니다\n퇴직금 지급시 irp계좌를 개설미적용되는 분을 정확이 알고 싶습니다 55세이 이하라고만 서치되어 있는데 만 55세이하분이 해당되는건지 궁금합니다또한 만 55세이하와 급여 300만원 이하조건의 동시에 해당되야지irp퇴직통장계좌 개설을 안해도 되는지 궁금합니다', 'Embedding': []}, '내담자_추가정보': '\n퇴직금 지급 관련 irp 통장 개설 조건을 알고 싶어 함\n', '해결방법': {'raw_text': '‘22. 4. 14.부터는 퇴직금을 근로자가 지정한 개인형퇴직연금제도의 계정 등으로 지급하여야 합니다.- 사용자는 가입자에게 퇴직금 수령을 위한 개인형 퇴직연금제도(IRP)계좌를 개설하도록 안내하고, 기한 내에 퇴직금을 세전 금액으로 지급해야 합니다.  다음의 예외 사유에 해당하는 경우에는 IRP계정 등으로 지급하지 않아도 됩니다.- 55세 이후 퇴직한 경우 - 퇴직급여액이 300만원 이하인 경우- 사망으로 인한 당연퇴직 및 외국인 근로자가 국외 출국한 경우- 타법령에서 퇴직소득을 공제할 수 있도록 한 경우  사용자는 퇴직금의 IRP계정 이전·지급 의무를 성실히 이행하여야 하며, 퇴직금을 근로자의 월급통장 등 일반계좌로 납입하는 것은 허용되지 않음.- 예외 사유에 해당하지 않는 한, 가입자의 신용불량만을 이유로 하여 IRP계정으로 지급하는 것을 거부할 수 없음. 퇴직금 중간정산은 주택구입 등 한정된 사유에 한하여 긴급한 생활자금이 필요한 근로자에게 퇴직금을 미리 정산하여 지급하는 것으로 중간정산 취지상 IRP계정으로 지급하지 않아도 됩니다.위의 예외사유중 한가지라도 충족한다면 IRP계정으로 지급하지 않아도 되며 55세는 만55세를 의미합니다.위 답변은 당시의 상황에 따른 것이며 법령 개정 등에 따라 변경될 수 있으며, 개인의 구체적인 사실에 따라 법규정해석 등에 이견이 있을 수 있습니다. 기타 궁금한 사항은 고객상담센터(1350)에 문의하시기 바랍니

In [11]:
data[0]

{'상담제목': {'raw_text': '퇴직금 지급받을 irp통장 조건을 확실히 알고싶습니다\n퇴직금 지급시 irp계좌를 개설미적용되는 분을 정확이 알고 싶습니다 55세이 이하라고만 서치되어 있는데 만 55세이하분이 해당되는건지 궁금합니다또한 만 55세이하와 급여 300만원 이하조건의 동시에 해당되야지irp퇴직통장계좌 개설을 안해도 되는지 궁금합니다',
  'Embedding': []},
 '내담자_추가정보': '\n퇴직금 지급 관련 irp 통장 개설 조건을 알고 싶어 함\n',
 '해결방법': {'raw_text': '‘22. 4. 14.부터는 퇴직금을 근로자가 지정한 개인형퇴직연금제도의 계정 등으로 지급하여야 합니다.- 사용자는 가입자에게 퇴직금 수령을 위한 개인형 퇴직연금제도(IRP)계좌를 개설하도록 안내하고, 기한 내에 퇴직금을 세전 금액으로 지급해야 합니다.  다음의 예외 사유에 해당하는 경우에는 IRP계정 등으로 지급하지 않아도 됩니다.- 55세 이후 퇴직한 경우 - 퇴직급여액이 300만원 이하인 경우- 사망으로 인한 당연퇴직 및 외국인 근로자가 국외 출국한 경우- 타법령에서 퇴직소득을 공제할 수 있도록 한 경우  사용자는 퇴직금의 IRP계정 이전·지급 의무를 성실히 이행하여야 하며, 퇴직금을 근로자의 월급통장 등 일반계좌로 납입하는 것은 허용되지 않음.- 예외 사유에 해당하지 않는 한, 가입자의 신용불량만을 이유로 하여 IRP계정으로 지급하는 것을 거부할 수 없음. 퇴직금 중간정산은 주택구입 등 한정된 사유에 한하여 긴급한 생활자금이 필요한 근로자에게 퇴직금을 미리 정산하여 지급하는 것으로 중간정산 취지상 IRP계정으로 지급하지 않아도 됩니다.위의 예외사유중 한가지라도 충족한다면 IRP계정으로 지급하지 않아도 되며 55세는 만55세를 의미합니다.위 답변은 당시의 상황에 따른 것이며 법령 개정 등에 따라 변경될 수 있으며, 개인의 구체적인 사실에 따라 법규정해석 등에 이견이 있을 수 있습니다. 기타 궁금한 사항은 고객상담센터(1350)에 문의하시기

In [12]:
data_list = []

for row in data:
    data_list.append({"title" : row['내담자_추가정보'],
    "contents" : "[상담내용]\n" + row["상담제목"]["raw_text"] + "\n\n[해결방법]\n" + row["해결방법"]["raw_text"],
    "metadata" : {"url" : row['metadata']['url'], "date" : row['metadata']['date']}})

In [13]:
data_list[0]

{'title': '\n퇴직금 지급 관련 irp 통장 개설 조건을 알고 싶어 함\n',
 'contents': '[상담내용]\n퇴직금 지급받을 irp통장 조건을 확실히 알고싶습니다\n퇴직금 지급시 irp계좌를 개설미적용되는 분을 정확이 알고 싶습니다 55세이 이하라고만 서치되어 있는데 만 55세이하분이 해당되는건지 궁금합니다또한 만 55세이하와 급여 300만원 이하조건의 동시에 해당되야지irp퇴직통장계좌 개설을 안해도 되는지 궁금합니다\n\n[해결방법]\n‘22. 4. 14.부터는 퇴직금을 근로자가 지정한 개인형퇴직연금제도의 계정 등으로 지급하여야 합니다.- 사용자는 가입자에게 퇴직금 수령을 위한 개인형 퇴직연금제도(IRP)계좌를 개설하도록 안내하고, 기한 내에 퇴직금을 세전 금액으로 지급해야 합니다.  다음의 예외 사유에 해당하는 경우에는 IRP계정 등으로 지급하지 않아도 됩니다.- 55세 이후 퇴직한 경우 - 퇴직급여액이 300만원 이하인 경우- 사망으로 인한 당연퇴직 및 외국인 근로자가 국외 출국한 경우- 타법령에서 퇴직소득을 공제할 수 있도록 한 경우  사용자는 퇴직금의 IRP계정 이전·지급 의무를 성실히 이행하여야 하며, 퇴직금을 근로자의 월급통장 등 일반계좌로 납입하는 것은 허용되지 않음.- 예외 사유에 해당하지 않는 한, 가입자의 신용불량만을 이유로 하여 IRP계정으로 지급하는 것을 거부할 수 없음. 퇴직금 중간정산은 주택구입 등 한정된 사유에 한하여 긴급한 생활자금이 필요한 근로자에게 퇴직금을 미리 정산하여 지급하는 것으로 중간정산 취지상 IRP계정으로 지급하지 않아도 됩니다.위의 예외사유중 한가지라도 충족한다면 IRP계정으로 지급하지 않아도 되며 55세는 만55세를 의미합니다.위 답변은 당시의 상황에 따른 것이며 법령 개정 등에 따라 변경될 수 있으며, 개인의 구체적인 사실에 따라 법규정해석 등에 이견이 있을 수 있습니다. 기타 궁금한 사항은 고객상담센터(1350)에 문의하시기 바랍니다. 감사합니다. \u200b',
 'metadata': {'

In [14]:
# JSON 파일로 저장
output_file = "./postprocessed_data/jisikin_2020_2024.json"
with open(output_file, 'w', encoding='utf-8') as file:
    json.dump(data_list, file, ensure_ascii=False, indent=4)

print(f"JSON 파일이 저장되었습니다: {output_file}")

JSON 파일이 저장되었습니다: ./postprocessed_data/jisikin_2020_2024.json
